In [0]:
from typing import Any
from types import NoneType

from dbruntime.dbutils import FileInfo


class SessionHelper:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super(SessionHelper, cls).__new__(cls)
        return cls._instance
    
    def __init__(self):
        self.storage_account_name = "catalogstorage33452"
        self.storage_container_name = "f1-project"
        self._set_storage_access_key()
        self._initialize_catalog()
        print("Session succesfully initialized!")

    def _initialize_catalog(self):
        spark.sql("CREATE CATALOG IF NOT EXISTS F1")
        spark.sql("USE CATALOG F1")
        spark.sql("CREATE DATABASE IF NOT EXISTS dev")
        spark.sql("CREATE DATABASE IF NOT EXISTS test")
        spark.sql("CREATE DATABASE IF NOT EXISTS prod")

    def _set_storage_access_key(self) -> None:
        storage_account_key = dbutils.secrets.get(
            scope="f1-project-secrets", key="f1-storage-access-key"
        )
        spark.conf.set(
            f"fs.azure.account.key.{self.storage_account_name}.dfs.core.windows.net",
            storage_account_key,
        )

    def get_storage_account_url(self, folder: str | NoneType = None, file: str | NoneType = None) -> str:
        folder = folder if folder is not None else ""
        file = "/" + file if file is not None else ""
        storage_account_url = f"abfss://{self.storage_container_name}@{self.storage_account_name}.dfs.core.windows.net/"
        return storage_account_url + folder + file

    def get_storage_account_files(
        self, folder: str | NoneType = None
    ) -> list[FileInfo]:
        storage_account_url = self.get_storage_account_url(folder)
        return dbutils.fs.ls(storage_account_url)


session_helper = SessionHelper()